In [1]:
!which python


/home/nandwani_vaibhav/anaconda3/envs/fastai/bin/python


In [2]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score
from torch.autograd import Variable
from torch.autograd import Function
import torch.backends.cudnn as cudnn
import os
import numpy as np
from tqdm import tqdm

#from eval_metrics import evaluate
from logger import Logger
# from TripletFaceDataset import TripletFaceDataset
# from LFWDataset import LFWDataset
from PIL import Image
#from utils import PairwiseDistance,display_triplet_distance,display_triplet_distance_test
import collections
from torchvision.datasets import ImageFolder
import math
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects

/home/nandwani_vaibhav/anaconda3/envs/fastai/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
dataroot = '/home/nandwani_vaibhav/wildlife/data/train/'
testpath = '/home/nandwani_vaibhav/wildlife/data/test/'
log_dir = '/home/nandwani_vaibhav/wildlife/primNet/logs/'

In [4]:

#resume
start_epoch = 0
epochs = 5
# Training options
embedding_size = 256
batch_size = 64
test_batch_size = 64
lr = 0.1
lr_decay = 1e-4
wd = 0.0
optimizer ='adagrad'
# Device options
no_cuda = False
gpu_id ='0'
seed = 0
log_interval = 10


# set the device to use by setting CUDA_VISIBLE_DEVICES env variable in
# order to prevent any memory allocation on unused GPUs
os.environ['CUDA_VISIBLE_DEVICES'] = gpu_id

cuda = not no_cuda and torch.cuda.is_available()
np.random.seed(seed)

# if not os.path.exists(log_dir):
#     os.makedirs(log_dir)

if cuda:
    cudnn.benchmark = True

LOG_DIR = log_dir 




In [5]:
# create logger
logger = Logger(LOG_DIR)

In [6]:
class Scale(object):
    """Rescales the input PIL.Image to the given 'size'.
    If 'size' is a 2-element tuple or list in the order of (width, height), it will be the exactly size to scale.
    If 'size' is a number, it will indicate the size of the smaller edge.
    For example, if height > width, then image will be
    rescaled to (size * height / width, size)
    size: size of the exactly size or the smaller edge
    interpolation: Default: PIL.Image.BILINEAR
    """

    def __init__(self, size, interpolation=Image.BILINEAR):
        assert isinstance(size, int) or (isinstance(size, collections.Iterable) and len(size) == 2)
        self.size = size
        self.interpolation = interpolation

    def __call__(self, img):
        if isinstance(self.size, int):
            w, h = img.size
            if (w <= h and w == self.size) or (h <= w and h == self.size):
                return img
            if w < h:
                ow = self.size
                oh = int(self.size * h / w)
                return img.resize((ow, oh), self.interpolation)
            else:
                oh = self.size
                ow = int(self.size * w / h)
                return img.resize((ow, oh), self.interpolation)
        else:
            return img.resize(self.size, self.interpolation)

In [7]:

def validation_iterator(dataLoader):
    for data, target in dataLoader:
        yield data, target        


In [8]:
kwargs = {'num_workers': 2, 'pin_memory': True} if cuda else {}
transform = transforms.Compose([
                         Scale((112,112)),
                         transforms.ToTensor(),
                         transforms.Normalize(mean = [ 0.5, 0.5, 0.5 ],
                                               std = [ 0.5, 0.5, 0.5 ])
                     ])


In [9]:
test_dataset = ImageFolder(testpath, transform=transform)
train_dataset = ImageFolder(dataroot, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, shuffle=True, **kwargs)
train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, **kwargs)
val_iterator = validation_iterator(test_loader)
train_iterator = validation_iterator(train_loader)

In [10]:
print(len(train_dataset.classes))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(len(train_dataset))
print(len(train_loader))
print(len(test_dataset))
print(len(test_loader))

89
5873
5873
1289
1289


In [11]:
## MODEL.py ##

import torch
import torch.nn as nn
from torchvision.models import resnet18
import math
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np

def channel_shuffle(x, groups):
    batchsize, num_channels, height, width = x.data.size()

    channels_per_group = num_channels // groups

    # reshape
    x = x.view(batchsize, groups, channels_per_group, height, width)

    # transpose
    # - contiguous() required if transpose() is used before view().
    #   See https://github.com/pytorch/pytorch/issues/764
    x = torch.transpose(x, 1, 2).contiguous()

    # flatten
    x = x.view(batchsize, -1, height, width)
    return x
    

class PrimNet(nn.Module):
    def __init__(self,embedding_size,num_classes,pretrained=False):
        super(PrimNet, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 256, kernel_size=3, stride=2, padding=1, groups = 32),
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer3 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=2, padding=1, groups = 32),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(512, 1024, kernel_size=3, stride=2, padding=1, groups = 32),
            nn.BatchNorm2d(1024),
            nn.ReLU())
        self.fc1 = nn.Linear(7*7*1024, embedding_size)
        self.fc2 = nn.Linear(embedding_size, num_classes)
    
   
    
    def l2_norm(self,input):
        input_size = input.size()
        buffer = torch.pow(input, 2)

        normp = torch.sum(buffer, 1).add_(1e-10)
        norm = torch.sqrt(normp)

        _output = torch.div(input, norm.view(-1, 1).expand_as(input))

        output = _output.view(input_size)

        return output
    
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = channel_shuffle(out, 32)
        out = self.layer3(out)
        out = channel_shuffle(out, 32)
        out = self.layer4(out)
        out = channel_shuffle(out, 32)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        self.features = self.l2_norm(out)
        # Multiply by alpha = 10 as suggested in https://arxiv.org/pdf/1703.09507.pdf
        alpha=10
        self.features = self.features*alpha
        return self.features
    
    def forward_classifier(self, x):
        features = self.forward(x)
        res = self.fc2(features)
        return res

In [12]:
# import model

In [13]:
net = PrimNet(embedding_size=embedding_size,
                      num_classes=len(train_dataset.classes),
                      pretrained=False).to(device)

In [14]:
print(torch.__version__)
torch.cuda.is_available()

0.4.0


True

In [15]:
# if args.resume:
#         if os.path.isfile(args.resume):
#             print('=> loading checkpoint {}'.format(args.resume))
#             checkpoint = torch.load(args.resume)
#             args.start_epoch = checkpoint['epoch']
#             checkpoint = torch.load(args.resume)
#             model.load_state_dict(checkpoint['state_dict'])
#         else:
#             print('=> no checkpoint found at {}'.format(args.resume))


In [17]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

# Train the model
total_step = len(train_loader)
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = net(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 1 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, epochs, i+1, total_step, loss.item()))
            
    torch.save({'epoch': epoch + 1, 'state_dict': net.state_dict()},
               '{}/checkpoint_{}.pth'.format(LOG_DIR, epoch))
    
        

Epoch [1/5], Step [1/5873], Loss: 3.3189
Epoch [1/5], Step [2/5873], Loss: 4.7692
Epoch [1/5], Step [3/5873], Loss: 4.6633
Epoch [1/5], Step [4/5873], Loss: 4.3682
Epoch [1/5], Step [5/5873], Loss: 4.8028
Epoch [1/5], Step [6/5873], Loss: 4.7255
Epoch [1/5], Step [7/5873], Loss: 4.7638
Epoch [1/5], Step [8/5873], Loss: 3.9401
Epoch [1/5], Step [9/5873], Loss: 4.2348
Epoch [1/5], Step [10/5873], Loss: 4.9853
Epoch [1/5], Step [11/5873], Loss: 4.8254
Epoch [1/5], Step [12/5873], Loss: 4.6027
Epoch [1/5], Step [13/5873], Loss: 5.3315
Epoch [1/5], Step [14/5873], Loss: 4.2478
Epoch [1/5], Step [15/5873], Loss: 4.9121
Epoch [1/5], Step [16/5873], Loss: 4.9252
Epoch [1/5], Step [17/5873], Loss: 4.5858
Epoch [1/5], Step [18/5873], Loss: 4.4825
Epoch [1/5], Step [19/5873], Loss: 4.9400
Epoch [1/5], Step [20/5873], Loss: 5.3266
Epoch [1/5], Step [21/5873], Loss: 4.4422
Epoch [1/5], Step [22/5873], Loss: 4.7196
Epoch [1/5], Step [23/5873], Loss: 3.8188
Epoch [1/5], Step [24/5873], Loss: 5.1380
E

Epoch [1/5], Step [205/5873], Loss: 4.3659
Epoch [1/5], Step [206/5873], Loss: 3.9992
Epoch [1/5], Step [207/5873], Loss: 4.0589
Epoch [1/5], Step [208/5873], Loss: 5.9578
Epoch [1/5], Step [209/5873], Loss: 3.9134
Epoch [1/5], Step [210/5873], Loss: 5.0040
Epoch [1/5], Step [211/5873], Loss: 3.5954
Epoch [1/5], Step [212/5873], Loss: 4.8574
Epoch [1/5], Step [213/5873], Loss: 4.4317
Epoch [1/5], Step [214/5873], Loss: 4.9772
Epoch [1/5], Step [215/5873], Loss: 4.2038
Epoch [1/5], Step [216/5873], Loss: 4.0788
Epoch [1/5], Step [217/5873], Loss: 4.7345
Epoch [1/5], Step [218/5873], Loss: 4.3962
Epoch [1/5], Step [219/5873], Loss: 3.8645
Epoch [1/5], Step [220/5873], Loss: 5.0162
Epoch [1/5], Step [221/5873], Loss: 4.8326
Epoch [1/5], Step [222/5873], Loss: 4.2788
Epoch [1/5], Step [223/5873], Loss: 4.4451
Epoch [1/5], Step [224/5873], Loss: 3.8472
Epoch [1/5], Step [225/5873], Loss: 2.6126
Epoch [1/5], Step [226/5873], Loss: 3.7385
Epoch [1/5], Step [227/5873], Loss: 4.5662
Epoch [1/5]

Epoch [1/5], Step [402/5873], Loss: 4.5869
Epoch [1/5], Step [403/5873], Loss: 5.2177
Epoch [1/5], Step [404/5873], Loss: 4.3431
Epoch [1/5], Step [405/5873], Loss: 4.8761
Epoch [1/5], Step [406/5873], Loss: 5.0402
Epoch [1/5], Step [407/5873], Loss: 5.8858
Epoch [1/5], Step [408/5873], Loss: 4.7356
Epoch [1/5], Step [409/5873], Loss: 4.7754
Epoch [1/5], Step [410/5873], Loss: 2.7832
Epoch [1/5], Step [411/5873], Loss: 4.1740
Epoch [1/5], Step [412/5873], Loss: 2.7878
Epoch [1/5], Step [413/5873], Loss: 3.4709
Epoch [1/5], Step [414/5873], Loss: 4.9337
Epoch [1/5], Step [415/5873], Loss: 4.7412
Epoch [1/5], Step [416/5873], Loss: 4.8966
Epoch [1/5], Step [417/5873], Loss: 5.7508
Epoch [1/5], Step [418/5873], Loss: 4.9414
Epoch [1/5], Step [419/5873], Loss: 4.5715
Epoch [1/5], Step [420/5873], Loss: 4.3554
Epoch [1/5], Step [421/5873], Loss: 4.8064
Epoch [1/5], Step [422/5873], Loss: 3.3533
Epoch [1/5], Step [423/5873], Loss: 4.7458
Epoch [1/5], Step [424/5873], Loss: 4.6322
Epoch [1/5]

Epoch [1/5], Step [600/5873], Loss: 6.2089
Epoch [1/5], Step [601/5873], Loss: 3.7181
Epoch [1/5], Step [602/5873], Loss: 5.4958
Epoch [1/5], Step [603/5873], Loss: 5.6453
Epoch [1/5], Step [604/5873], Loss: 5.1738
Epoch [1/5], Step [605/5873], Loss: 4.5670
Epoch [1/5], Step [606/5873], Loss: 3.7497
Epoch [1/5], Step [607/5873], Loss: 3.3086
Epoch [1/5], Step [608/5873], Loss: 4.3189
Epoch [1/5], Step [609/5873], Loss: 3.8634
Epoch [1/5], Step [610/5873], Loss: 5.3895
Epoch [1/5], Step [611/5873], Loss: 5.1450
Epoch [1/5], Step [612/5873], Loss: 5.1879
Epoch [1/5], Step [613/5873], Loss: 4.1889
Epoch [1/5], Step [614/5873], Loss: 4.2896
Epoch [1/5], Step [615/5873], Loss: 4.1794
Epoch [1/5], Step [616/5873], Loss: 3.7118
Epoch [1/5], Step [617/5873], Loss: 3.2044
Epoch [1/5], Step [618/5873], Loss: 4.9006
Epoch [1/5], Step [619/5873], Loss: 3.5021
Epoch [1/5], Step [620/5873], Loss: 4.8355
Epoch [1/5], Step [621/5873], Loss: 4.7506
Epoch [1/5], Step [622/5873], Loss: 4.0225
Epoch [1/5]

Epoch [1/5], Step [792/5873], Loss: 4.1370
Epoch [1/5], Step [793/5873], Loss: 3.1510
Epoch [1/5], Step [794/5873], Loss: 3.9247
Epoch [1/5], Step [795/5873], Loss: 5.0302
Epoch [1/5], Step [796/5873], Loss: 3.8588
Epoch [1/5], Step [797/5873], Loss: 3.8763
Epoch [1/5], Step [798/5873], Loss: 4.8156
Epoch [1/5], Step [799/5873], Loss: 3.3318
Epoch [1/5], Step [800/5873], Loss: 5.6403
Epoch [1/5], Step [801/5873], Loss: 3.1241
Epoch [1/5], Step [802/5873], Loss: 5.5389
Epoch [1/5], Step [803/5873], Loss: 3.6294
Epoch [1/5], Step [804/5873], Loss: 4.5215
Epoch [1/5], Step [805/5873], Loss: 3.7976
Epoch [1/5], Step [806/5873], Loss: 4.7670
Epoch [1/5], Step [807/5873], Loss: 3.9790
Epoch [1/5], Step [808/5873], Loss: 3.0657
Epoch [1/5], Step [809/5873], Loss: 4.3207
Epoch [1/5], Step [810/5873], Loss: 4.4873
Epoch [1/5], Step [811/5873], Loss: 5.3551
Epoch [1/5], Step [812/5873], Loss: 3.6091
Epoch [1/5], Step [813/5873], Loss: 5.5416
Epoch [1/5], Step [814/5873], Loss: 3.9114
Epoch [1/5]

...

Epoch [1/5], Step [5144/5873], Loss: 3.5931
Epoch [1/5], Step [5145/5873], Loss: 5.1470
Epoch [1/5], Step [5146/5873], Loss: 4.3116
Epoch [1/5], Step [5147/5873], Loss: 4.1304
Epoch [1/5], Step [5148/5873], Loss: 5.3373
Epoch [1/5], Step [5149/5873], Loss: 2.7321
Epoch [1/5], Step [5150/5873], Loss: 1.9206
Epoch [1/5], Step [5151/5873], Loss: 5.7218
Epoch [1/5], Step [5152/5873], Loss: 3.6941
Epoch [1/5], Step [5153/5873], Loss: 4.5829
Epoch [1/5], Step [5154/5873], Loss: 3.9107
Epoch [1/5], Step [5155/5873], Loss: 4.2911
Epoch [1/5], Step [5156/5873], Loss: 4.3861
Epoch [1/5], Step [5157/5873], Loss: 3.3745
Epoch [1/5], Step [5158/5873], Loss: 2.8983
Epoch [1/5], Step [5159/5873], Loss: 4.7025
Epoch [1/5], Step [5160/5873], Loss: 1.3852
Epoch [1/5], Step [5161/5873], Loss: 3.4800
Epoch [1/5], Step [5162/5873], Loss: 3.2031
Epoch [1/5], Step [5163/5873], Loss: 3.8391
Epoch [1/5], Step [5164/5873], Loss: 2.9723
Epoch [1/5], Step [5165/5873], Loss: 4.2670
Epoch [1/5], Step [5166/5873], L

Epoch [1/5], Step [5339/5873], Loss: 3.8748
Epoch [1/5], Step [5340/5873], Loss: 4.3252
Epoch [1/5], Step [5341/5873], Loss: 3.9608
Epoch [1/5], Step [5342/5873], Loss: 3.3559
Epoch [1/5], Step [5343/5873], Loss: 3.5949
Epoch [1/5], Step [5344/5873], Loss: 3.8413
Epoch [1/5], Step [5345/5873], Loss: 2.9386
Epoch [1/5], Step [5346/5873], Loss: 4.9091
Epoch [1/5], Step [5347/5873], Loss: 5.2334
Epoch [1/5], Step [5348/5873], Loss: 4.8487
Epoch [1/5], Step [5349/5873], Loss: 4.5994
Epoch [1/5], Step [5350/5873], Loss: 2.1446
Epoch [1/5], Step [5351/5873], Loss: 2.6963
Epoch [1/5], Step [5352/5873], Loss: 2.6005
Epoch [1/5], Step [5353/5873], Loss: 2.2863
Epoch [1/5], Step [5354/5873], Loss: 3.5004
Epoch [1/5], Step [5355/5873], Loss: 3.3129
Epoch [1/5], Step [5356/5873], Loss: 6.6807
Epoch [1/5], Step [5357/5873], Loss: 3.1607
Epoch [1/5], Step [5358/5873], Loss: 3.7619
Epoch [1/5], Step [5359/5873], Loss: 3.9338
Epoch [1/5], Step [5360/5873], Loss: 2.7754
Epoch [1/5], Step [5361/5873], L

Epoch [1/5], Step [5527/5873], Loss: 4.1307
Epoch [1/5], Step [5528/5873], Loss: 4.1778
Epoch [1/5], Step [5529/5873], Loss: 5.2667
Epoch [1/5], Step [5530/5873], Loss: 5.6405
Epoch [1/5], Step [5531/5873], Loss: 2.6804
Epoch [1/5], Step [5532/5873], Loss: 2.3353
Epoch [1/5], Step [5533/5873], Loss: 4.4115
Epoch [1/5], Step [5534/5873], Loss: 3.0846
Epoch [1/5], Step [5535/5873], Loss: 3.7709
Epoch [1/5], Step [5536/5873], Loss: 3.5342
Epoch [1/5], Step [5537/5873], Loss: 3.1906
Epoch [1/5], Step [5538/5873], Loss: 4.5236
Epoch [1/5], Step [5539/5873], Loss: 3.9120
Epoch [1/5], Step [5540/5873], Loss: 4.4096
Epoch [1/5], Step [5541/5873], Loss: 1.2179
Epoch [1/5], Step [5542/5873], Loss: 5.1771
Epoch [1/5], Step [5543/5873], Loss: 5.0115
Epoch [1/5], Step [5544/5873], Loss: 6.3902
Epoch [1/5], Step [5545/5873], Loss: 3.0542
Epoch [1/5], Step [5546/5873], Loss: 5.3144
Epoch [1/5], Step [5547/5873], Loss: 3.8130
Epoch [1/5], Step [5548/5873], Loss: 4.1936
Epoch [1/5], Step [5549/5873], L

Epoch [1/5], Step [5716/5873], Loss: 3.5579
Epoch [1/5], Step [5717/5873], Loss: 5.0966
Epoch [1/5], Step [5718/5873], Loss: 2.6267
Epoch [1/5], Step [5719/5873], Loss: 3.3463
Epoch [1/5], Step [5720/5873], Loss: 2.2856
Epoch [1/5], Step [5721/5873], Loss: 3.4236
Epoch [1/5], Step [5722/5873], Loss: 4.3669
Epoch [1/5], Step [5723/5873], Loss: 4.7801
Epoch [1/5], Step [5724/5873], Loss: 3.4438
Epoch [1/5], Step [5725/5873], Loss: 4.9850
Epoch [1/5], Step [5726/5873], Loss: 3.0831
Epoch [1/5], Step [5727/5873], Loss: 4.2125
Epoch [1/5], Step [5728/5873], Loss: 1.9567
Epoch [1/5], Step [5729/5873], Loss: 3.7752
Epoch [1/5], Step [5730/5873], Loss: 2.8635
Epoch [1/5], Step [5731/5873], Loss: 1.8707
Epoch [1/5], Step [5732/5873], Loss: 4.5623
Epoch [1/5], Step [5733/5873], Loss: 3.2887
Epoch [1/5], Step [5734/5873], Loss: 2.7552
Epoch [1/5], Step [5735/5873], Loss: 4.8043
Epoch [1/5], Step [5736/5873], Loss: 1.4101
Epoch [1/5], Step [5737/5873], Loss: 3.5591
Epoch [1/5], Step [5738/5873], L

ValueError: need at least one array to concatenate

In [ ]:
torch.save(net, 'mytraining.pt')



In [ ]:
torch.load('mytraining.pt')

In [ ]:
##TESTING##
net.eval()


In [21]:
print('extracting feature')
embeds = []
labels = []
preds = np.zeros((1289,1))
j = 0
for data, target in test_loader:
    if cuda:
        data, target = data.cuda(), target.cuda(async=True)  
    data_var = Variable(data, volatile=True)
    # compute output
    output = net(data_var)
    print(j)
    cls = net.forward_classifier(data_var)
    values, indices = torch.max(cls, 1)
    indices = indices + 1
#   print(target.size())
    embeds.append( output.data.cpu().numpy() )
    labels.append( target.cpu().numpy() )
    indices = np.array(indices)
    for i in range(indices.shape[0]):
            preds[i + j] = indices[i]
    j = j + indices.shape[0]

extracting feature
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


/home/nandwani_vaibhav/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  if __name__ == '__main__':


17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287

In [22]:
embeds = np.vstack(embeds)
labels = np.hstack(labels)
print(preds.shape)
print(preds)
print(labels)
print('embeds shape is ', embeds.shape)
print('labels shape is ', labels.shape)
print('Test Accuracy: ',accuracy_score(labels, preds))

(1289, 1)
[[59.]
 [59.]
 [32.]
 ...
 [74.]
 [45.]
 [81.]]
[44 43  3 ... 42  2 78]
embeds shape is  (1289, 256)
labels shape is  (1289,)
Test Accuracy:  0.008533747090768037


In [ ]:
print('extracting feature train')
embeds_t = []
labels_t = []
preds_t = np.zeros((5873,1))
j = 0
for data, target in train_iterator:
    if cuda:
        data, target = data.cuda(), target.cuda(async=True)
        
    data_var = Variable(data, volatile=True)
    # compute output
    output = net(data_var)
    cls = net.forward_classifier(data_var)
    values, indices = torch.max(cls, 1)
    indices = indices
#         print(target.size())
    embeds_t.append( output.data.cpu().numpy() )
    labels_t.append( target.cpu().numpy() )
    indices = np.array(indices)
    for i in range(indices.shape[0]):
           preds_t[i + j] = indices[i]
    j = j + indices.shape[0]
        
        
    


In [ ]:
#preds = np.hstack(preds)
embeds_t = np.vstack(embeds_t)
labels_t = np.hstack(labels_t)
#     print(preds_t.shape)
#     print(preds_t)
#     print(labels_t)
print('embeds shape is ', embeds_t.shape)
print('labels shape is ', labels_t.shape)
print('Train accyracy: ',accuracy_score(labels_t, preds_t))